In [6]:
import sys
import os

# Get current notebook directory
current_dir = os.getcwd()

# Add parent directory
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
import peptide

In [47]:
def create_fake_pairs(the_peptide):
    the_pep = peptide.Pep(the_peptide)
    pair_result = []
    result = []
    for i in range(1, len(the_pep.AA_array)):
        frag1 = the_pep.AA_array[:i]
        frag2 = the_pep.AA_array[i:]
        mass1 = sum([i.get_mass() for i in frag1])
        mass2 = sum([i.get_mass() for i in frag2])
        mass2 += 18.01056  # Adding H2O mass to the second fragment
        print(f"Fragment 1: {frag1}, Mass: {mass1:.4f}")
        print(f"Fragment 2: {frag2}, Mass: {mass2:.4f}")
        pair_result.append([mass1, mass2]) 
        result.append(mass1)
        result.append(mass2)
    result.sort()
    
    return result, pair_result

In [60]:
seq = "[CGEYFQ+2H]2+"
sorted_array, pair_array = create_fake_pairs(seq)

Fragment 1: [C], Mass: 103.0092
Fragment 2: [G, E, Y, F, Q], Mass: 642.2649
Fragment 1: [C, G], Mass: 160.0307
Fragment 2: [E, Y, F, Q], Mass: 585.2435
Fragment 1: [C, G, E], Mass: 289.0732
Fragment 2: [Y, F, Q], Mass: 456.2009
Fragment 1: [C, G, E, Y], Mass: 452.1366
Fragment 2: [F, Q], Mass: 293.1375
Fragment 1: [C, G, E, Y, F], Mass: 599.2050
Fragment 2: [Q], Mass: 146.0691


In [61]:
sorted_array = [0] + sorted_array + [sum(pair_array[0])]

In [62]:
sorted_array

[0,
 103.00919,
 146.06914,
 160.03065,
 289.07324,
 293.13755,
 452.13657,
 456.20088000000004,
 585.24347,
 599.20498,
 642.26493,
 745.27412]

In [63]:
mid_point = len(sorted_array) // 2
lower_half = sorted_array[:mid_point]
upper_half = sorted_array[mid_point:]
len(lower_half), len(upper_half)

(6, 6)

In [64]:
def generate_psp_grid(spectrum):
    # Ensure the spectrum is sorted to match the index logic
    S = sorted(list(spectrum))
    n = len(S)
    
    # Create an n x n grid
    # We iterate rows (i) then columns (j)
    # Label logic: (Value of Column j, Value of Row i)
    grid = [[(S[j], S[i]) for j in range(n)] for i in range(n)]
    
    return grid


In [65]:
generate_psp_grid(lower_half)

[[(0, 0),
  (103.00919, 0),
  (146.06914, 0),
  (160.03065, 0),
  (289.07324, 0),
  (293.13755, 0)],
 [(0, 103.00919),
  (103.00919, 103.00919),
  (146.06914, 103.00919),
  (160.03065, 103.00919),
  (289.07324, 103.00919),
  (293.13755, 103.00919)],
 [(0, 146.06914),
  (103.00919, 146.06914),
  (146.06914, 146.06914),
  (160.03065, 146.06914),
  (289.07324, 146.06914),
  (293.13755, 146.06914)],
 [(0, 160.03065),
  (103.00919, 160.03065),
  (146.06914, 160.03065),
  (160.03065, 160.03065),
  (289.07324, 160.03065),
  (293.13755, 160.03065)],
 [(0, 289.07324),
  (103.00919, 289.07324),
  (146.06914, 289.07324),
  (160.03065, 289.07324),
  (289.07324, 289.07324),
  (293.13755, 289.07324)],
 [(0, 293.13755),
  (103.00919, 293.13755),
  (146.06914, 293.13755),
  (160.03065, 293.13755),
  (289.07324, 293.13755),
  (293.13755, 293.13755)]]

In [68]:
import pandas as pd

def generate_psp_dataframe(spectrum):
    S = sorted(list(spectrum))
    n = len(S)
    
    # Generate data with the logic (S[col], S[row])
    data = [[(S[j], S[i]) for j in range(n)] for i in range(n)]
    
    # Create DataFrame with headers for clarity
    df = pd.DataFrame(data, index=S, columns=S)
    
    return df

df = generate_psp_dataframe(lower_half)
df

,0.00000,103.00919,146.06914,160.03065,289.07324,293.13755
0.00000,"(0, 0)","(103.00919, 0)","(146.06914, 0)","(160.03065, 0)","(289.07324, 0)","(293.13755, 0)"
103.00919,"(0, 103.00919)","(103.00919, 103.00919)","(146.06914, 103.00919)","(160.03065, 103.00919)","(289.07324, 103.00919)","(293.13755, 103.00919)"
146.06914,"(0, 146.06914)","(103.00919, 146.06914)","(146.06914, 146.06914)","(160.03065, 146.06914)","(289.07324, 146.06914)","(293.13755, 146.06914)"
160.03065,"(0, 160.03065)","(103.00919, 160.03065)","(146.06914, 160.03065)","(160.03065, 160.03065)","(289.07324, 160.03065)","(293.13755, 160.03065)"
289.07324,"(0, 289.07324)","(103.00919, 289.07324)","(146.06914, 289.07324)","(160.03065, 289.07324)","(289.07324, 289.07324)","(293.13755, 289.07324)"
293.13755,"(0, 293.13755)","(103.00919, 293.13755)","(146.06914, 293.13755)","(160.03065, 293.13755)","(289.07324, 293.13755)","(293.13755, 293.13755)"
